In [1]:
import requests
import pandas as pd
import datetime

In [5]:
import requests

api_key = "fjbIvblscE9aaoZhczJxUaZQuJAeGtsu" 
from urllib.parse import quote

def get_coordinates(address, api_key):
    encoded_address = quote(address)
    url = f"https://api.tomtom.com/search/2/geocode/{encoded_address}.json?key={api_key}"
    
    response = requests.get(url)
    data = response.json()

    if data["results"]:
        position = data["results"][0]["position"]
        lat = position["lat"]
        lon = position["lon"]
        return lat, lon
    else:
        return None, None

address = "15 Bhaktivedanta Swami Road, Vile Parle West, Mumbai, Maharashtra 400056"


lat, lon = get_coordinates(address, api_key)
print(f"Latitude: {lat}, Longitude: {lon}")


Latitude: 19.0989941, Longitude: 72.8395499


In [2]:
import requests
from datetime import datetime

def get_traffic_flow(lat, lon, api_key):
    url = "https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json"
    params = {
        "point": f"{lat},{lon}",
        "key": api_key
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        print(data)

        segment = data['flowSegmentData']

        current_speed = segment['currentSpeed']
        free_flow_speed = segment['freeFlowSpeed']
        confidence = segment['confidence']
        road_class = segment['frc']
        road_closed = segment['roadClosure']
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Compute congestion ratio
        congestion_ratio = round(current_speed / free_flow_speed, 2) if free_flow_speed else 0

        print("🚗 Traffic Flow Info:")
        print(f"- Timestamp:        {timestamp}")
        print(f"- Current Speed:    {current_speed} km/h")
        print(f"- Free Flow Speed:  {free_flow_speed} km/h")
        print(f"- Congestion Ratio: {congestion_ratio}")
        print(f"- Confidence:       {confidence}")
        print(f"- Road Class:       {road_class}")
        print(f"- Road Closed:      {road_closed}")

        return {
            "timestamp": timestamp,
            "lat": lat,
            "lon": lon,
            "current_speed": current_speed,
            "free_flow_speed": free_flow_speed,
            "congestion_ratio": congestion_ratio,
            "confidence": confidence,
            "road_class": road_class,
            "road_closed": road_closed
        }

    else:
        print("❌ Error fetching traffic data:", response.status_code)
        return None

# === Example usage ===
lat = 19.0989941
lon = 72.8395499
api_key = "fjbIvblscE9aaoZhczJxUaZQuJAeGtsu"

traffic_data = get_traffic_flow(lat, lon, api_key)


{'flowSegmentData': {'frc': 'FRC4', 'currentSpeed': 22, 'freeFlowSpeed': 22, 'currentTravelTime': 3250, 'freeFlowTravelTime': 3250, 'confidence': 1, 'roadClosure': False, 'coordinates': {'coordinate': [{'latitude': 19.25610829607037, 'longitude': 72.85819259771571}, {'latitude': 19.256096189228725, 'longitude': 72.85816309341655}, {'latitude': 19.256082816311093, 'longitude': 72.8581470001624}, {'latitude': 19.25606540777763, 'longitude': 72.85813493022184}, {'latitude': 19.255982242440073, 'longitude': 72.85810676702718}, {'latitude': 19.255833399328143, 'longitude': 72.858074580519}, {'latitude': 19.255730134888832, 'longitude': 72.85806385168294}, {'latitude': 19.255658997126147, 'longitude': 72.8580558050559}, {'latitude': 19.25559600971541, 'longitude': 72.85804909953336}, {'latitude': 19.25533179467246, 'longitude': 72.8580195952342}, {'latitude': 19.255219113434546, 'longitude': 72.85800752529363}, {'latitude': 19.255089023482093, 'longitude': 72.85798874983053}, {'latitude': 19

In [7]:
import sqlite3
def create_traffic_table(db_name="traffic_data.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS traffic_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            location_name TEXT,
            lat REAL,
            lon REAL,
            timestamp TEXT,
            current_speed REAL,
            free_flow_speed REAL,
            confidence REAL,
            road_class TEXT
        );
    """)

    conn.commit()
    conn.close()
    print("✅ SQLite table `traffic_data` created successfully.")
create_traffic_table()
    
    

✅ SQLite table `traffic_data` created successfully.


In [8]:
import sqlite3
from datetime import datetime

def insert_traffic_data(location_name, lat, lon, current_speed, free_flow_speed, confidence, road_class, db_name="traffic_data.db"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    timestamp = datetime.now().isoformat()

    cursor.execute("""
        INSERT INTO traffic_data (
            location_name, lat, lon, timestamp,
            current_speed, free_flow_speed, confidence, road_class
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """, (location_name, lat, lon, timestamp,
          current_speed, free_flow_speed, confidence, road_class))

    conn.commit()
    conn.close()
    print(f"✅ Data inserted for {location_name} at {timestamp}")


In [ ]:
import time
import requests

# TomTom API key
API_KEY = api_key

# List of diverse locations (Mumbai + Navi Mumbai)
locations = [
    {"name": "Dadar",     "lat": 19.0176, "lon": 72.8562},
    {"name": "Andheri",   "lat": 19.1197, "lon": 72.8468},
    {"name": "Powai",     "lat": 19.1197, "lon": 72.9051},
    {"name": "Colaba",    "lat": 18.9067, "lon": 72.8147},
    {"name": "Panvel",    "lat": 18.9894, "lon": 73.1175},
    {"name": "Airoli",    "lat": 19.1551, "lon": 72.9960},
    {"name": "Vashi",     "lat": 19.0760, "lon": 72.9986},
    {"name": "Nerul",     "lat": 19.0334, "lon": 73.0186},
    {"name": "Kharghar",  "lat": 19.0330, "lon": 73.0633}
]

# Fetch and insert traffic data for all locations
def fetch_and_store_all_locations():
    for loc in locations:
        try:
            url = f"https://api.tomtom.com/traffic/services/4/flowSegmentData/relative0/10/json"
            params = {
                "point": f"{loc['lat']},{loc['lon']}",
                "key": API_KEY
            }
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()["flowSegmentData"]

            insert_traffic_data(
                location_name = loc["name"],
                lat = loc["lat"],
                lon = loc["lon"],
                current_speed = data["currentSpeed"],
                free_flow_speed = data["freeFlowSpeed"],
                confidence = data["confidence"],
                road_class = data.get("roadClass", "Unknown")
            )

            print("done")

        except Exception as e:
            print(f"❌ Failed for {loc['name']}: {e}")

# Run every 15 minutes
if __name__ == "__main__":
    while True:
        print("\n⏳ Fetching traffic data for all locations...")
        fetch_and_store_all_locations()
        print("✅ All locations fetched and stored. Waiting 15 minutes...\n")
        time.sleep(15 * 60)  # Wait for 15 minutes (900 seconds)
